## Print @count statistics from network
Helmet 4.0 testing:
* Get network attributes
* Print statistics and barplots

Author: Atte Supponen, WSP, 2020

In [322]:
from IPython.display import display, HTML, Image
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Connect Emme-project, emmebank, scenario and network
_m = inro.modeller
modeller = _m.Modeller()
desktop = modeller.desktop
emmebank = modeller.emmebank
data_explorer = desktop.data_explorer()

# Emme tools
create_attr = modeller.tool("inro.emme.data.extra_attribute.create_extra_attribute")
import_attr = modeller.tool("inro.emme.data.extra_attribute.import_extra_attributes")


# Palettes fro plotting
palette1 = ['#97C798', '#B8F4BA', '#B9E5F4', '#F7B7B8']
palette2 = ['#42a600', '#56d900', '#0042A6', '#a60042']


### Read input obs data

In [323]:
scenarios = [21, 22, 23]
scenarios_filenames = ["links_2016_aht.txt", "links_2016_pt.txt", "links_2016_iht.txt"]

for i, j in zip(scenarios, scenarios_filenames):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@l_count",
                extra_attribute_description="observed counts",
                overwrite=True)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@m_count",
                extra_attribute_description="observed counts",
                overwrite=True)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@l_count_type",
                extra_attribute_description="observed counts types",
                overwrite=True)
    create_attr(extra_attribute_type="LINK",
                extra_attribute_name="@direction",
                extra_attribute_description="observed counts direction",
                overwrite=True)

    default_path = os.path.dirname(_m.Modeller().emmebank.path).replace("\\","/")

    file_path = os.path.join(default_path, "../Scripts/notebook/", j).replace("\\","/")

    import_attr(file_path,
                scenario = scenario_selection,
                field_separator=",",
                column_labels={0: "inode", 1: "jnode", 2: "@l_count", 3: "@l_count_type", 4: "@m_count", 5: "@direction"},
                revert_on_error=True)

In [324]:
directions_names = {
    "direction": [1,2,3,4],
    "direction_name": ["keskustaan","muualle","lanteen","itaan"]
}

directions_names = pd.DataFrame(data = directions_names)

#directions_names = directions_names.set_index('direction')

In [325]:
l_count_names = {
    "l_count_type":[1,2,3,4,5,6,7],
    "l_count_name":["Niemen raja", "Kantakaupungin raja", "Kaupungin raja", "Poikittaislinja", "Keha I", "Keha II", "Keha III"]
}

l_count_names = pd.DataFrame(data = l_count_names)

#l_count_names = l_count_names.set_index('l_count_type')

## L_COUNT Peak hours


In [326]:
# parameters
scenarios_ht = [21, 23]
scenarios_names = [u"2016_aht", u"2016_iht"]

scenarios = []
ids = []
l_counts = []
l_counts_types = []
directions = []

car_leisure = []
car_work = []
truck = []
trailer_truck = []
van = []
bus = []


for i, j in zip(scenarios_ht, scenarios_names):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    scenario = modeller.scenario
    network = scenario.get_network()
    #
    # list @code extra attribuuttiin tallennettuista vaylaosan tunnuksista
    for link in network.links():
        if link['@l_count'] > 0:
            scenarios.append(j)
            ids.append(link.id)
            l_counts.append(link['@l_count'])
            l_counts_types.append(link['@l_count_type'])
            directions.append(link['@direction'])
            
            car_leisure.append(link['@car_leisure'])
            car_work.append(link['@car_work'])
            truck.append(link['@truck'])
            trailer_truck.append(link['@trailer_truck'])
            van.append(link['@van'])
            bus.append(link['@bus'])

    l_sum = (np.array(car_leisure) + 
             np.array(car_work) + 
             np.array(truck) + 
             np.array(trailer_truck) + 
             np.array(van) +
             np.array(bus)
            )
    
d = {'scenario': scenarios,
     'id': ids, 
     'l_count': l_counts, 
     'l_count_type': l_counts_types, 
     'volau_ul3': l_sum, 
     'direction': directions} 
            
df = pd.DataFrame(data = d) 
df.to_csv("l_count_model.txt", index = False, sep='\t')

In [327]:
df = df.merge(l_count_names, on = "l_count_type", how='left')
df = df.merge(directions_names, on = "direction", how='left')

In [328]:
dfPlot = df.groupby(['scenario','direction_name','l_count_name'])['l_count','volau_ul3'].sum()
dfPlot = dfPlot.reset_index()

In [329]:
image_export_path = os.path.join("./Output/")
if not os.path.exists(image_export_path):
    os.mkdir(image_export_path)

for scenario in df["scenario"].unique():
    for direction in df["direction_name"].unique():
        
        plot_data = dfPlot.loc[dfPlot.scenario == scenario]
        plot_data = plot_data[dfPlot.direction_name == direction]
        
        fig, ax = plt.subplots(figsize = [9,5], dpi = 300)

        # set width of bar
        barWidth = 0.3
 
        # set height of bar
        bars_count = plot_data.filter(['l_count']).values
        bars_volau = plot_data.filter(['volau_ul3']).values
        
        # Set position of bar on X axis
        r1 = np.arange(len(bars_count))
        r2 = [x + barWidth for x in r1]

        # Make the plot
        ax_bars_count = ax.bar(r1, bars_count, color=palette1[0], width=barWidth, edgecolor='white', label=u'Laskenta', alpha = 0.9)
        ax_bars_volau = ax.bar(r2, bars_volau, color=palette1[1], width=barWidth, edgecolor='white', label=u'Mallinnus', alpha = 0.9)
        
        # Add xticks on the middle of the group bars
        plt.xticks([r + barWidth*0.5 for r in range(len(bars_count))], plot_data["l_count_name"])
        plt.tick_params(axis='both', which='major', labelsize=11)
        
        # axis names
        plt.xlim(-0.3, len(bars_count))
        plt.ylim(0, 50000) 
        plt.ylabel(u'Moottoriajoneuvoa', fontsize = 11)
        
        title = str(scenario).replace('_',' ') + ' ' + str(direction)
        plt.title(u'Liikennemäärät %s' % (title),fontweight="bold")
        
        # spines invisible
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        plt.tick_params(top='off', right='off', labelleft='on', labelbottom='on')
        
        # Shrink current axis's height by 10% on the bottom
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.15,
                         box.width, box.height * 0.85])
        
        # Put a legend below current axis
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon= False, fontsize = 11)
        
        def autolabel(bars):
            """
            Attach a text label above each bar displaying its height
            """
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., 1+height,
                        '%d' % int(round(height, -2)),
                        ha='center', va='bottom', color="black", fontsize=11)

        autolabel(ax_bars_count)
        autolabel(ax_bars_volau)
        
        filename = str(scenario) + '_' + str(direction)
        
        plt.savefig('./Output/%s_trips.png' % (filename.strip()))
        plt.close()
        

C:\Program Files\INRO\Emme\Emme 4\Emme-4.4.2\Python27\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


## Vuorokausi

In [330]:
# parameters
scenarios_ht = [22]
scenarios_names = [u"2016_vrk"]

scenarios = []
ids = []
l_counts = []
l_counts_types = []
directions = []

car_leisure = []
car_work = []
truck = []
trailer_truck = []
van = []
bus = []


for i, j in zip(scenarios_ht, scenarios_names):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    scenario = modeller.scenario
    network = scenario.get_network()
    #
    # list @code extra attribuuttiin tallennettuista vaylaosan tunnuksista
    for link in network.links():
        if link['@l_count'] > 0:
            scenarios.append(j)
            ids.append(link.id)
            l_counts.append(link['@l_count'])
            l_counts_types.append(link['@l_count_type'])
            directions.append(link['@direction'])
            
            car_leisure.append(link['@car_leisure_day'])
            car_work.append(link['@car_work_day'])
            truck.append(link['@truck_day'])
            trailer_truck.append(link['@trailer_truck_day'])
            van.append(link['@van_day'])
            bus.append(link['@bus'])

    l_sum = (np.array(car_leisure) + 
             np.array(car_work) + 
             np.array(truck) + 
             np.array(trailer_truck) + 
             np.array(van) +
             np.array(bus) * 2 * 6 + np.array(bus) * 10 
            )
    
d = {'scenario': scenarios,
     'id': ids, 
     'l_count': l_counts, 
     'l_count_type': l_counts_types, 
     'volau_ul3': l_sum, 
     'direction': directions} 
            
df = pd.DataFrame(data = d) 
df.to_csv("l_count_model.txt", index = False, sep='\t')

In [331]:
df = df.merge(l_count_names, on = "l_count_type", how='left')
df = df.merge(directions_names, on = "direction", how='left')

In [332]:
dfPlot = df.groupby(['scenario','direction_name','l_count_name'])['l_count','volau_ul3'].sum()
dfPlot = dfPlot.reset_index()

In [333]:
image_export_path = os.path.join("./Output/")
if not os.path.exists(image_export_path):
    os.mkdir(image_export_path)

for scenario in df["scenario"].unique():
    for direction in df["direction_name"].unique():
        
        plot_data = dfPlot.loc[dfPlot.scenario == scenario]
        plot_data = plot_data[dfPlot.direction_name == direction]
        
        fig, ax = plt.subplots(figsize = [9,5], dpi = 300)

        # set width of bar
        barWidth = 0.3
 
        # set height of bar
        bars_count = plot_data.filter(['l_count']).values
        bars_volau = plot_data.filter(['volau_ul3']).values

        # Set position of bar on X axis
        r1 = np.arange(len(bars_count))
        r2 = [x + barWidth for x in r1]

        # Make the plot
        ax_bars_count = ax.bar(r1, bars_count, color=palette1[0], width=barWidth, edgecolor='white', label=u'Laskenta', alpha = 0.9)
        ax_bars_volau = ax.bar(r2, bars_volau, color=palette1[1], width=barWidth, edgecolor='white', label=u'Mallinnus', alpha = 0.9)
        
        # Add xticks on the middle of the group bars
        plt.xticks([r + barWidth*0.5 for r in range(len(bars_count))], plot_data["l_count_name"])
        plt.tick_params(axis='both', which='major', labelsize=11)
        
        # axis names
        plt.xlim(-0.3, len(bars_count))
        plt.ylim(0, 500000) 
        plt.ylabel(u'Moottoriajoneuvoa', fontsize = 11)
        
        title = str(scenario).replace('_',' ') + ' ' + str(direction)
        plt.title(u'Liikennemäärät %s' % (title),fontweight="bold")
        
        # spines invisible
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        plt.tick_params(top='off', right='off', labelleft='on', labelbottom='on')
        
        # Shrink current axis's height by 10% on the bottom
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.15,
                         box.width, box.height * 0.85])
        
        # Put a legend below current axis
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon= False, fontsize = 11)
        
        def autolabel(bars):
            """
            Attach a text label above each bar displaying its height
            """
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., 1+height,
                        '%d' % int(round(height, -2)),
                        ha='center', va='bottom', color="black", fontsize=11)

        autolabel(ax_bars_count)
        autolabel(ax_bars_volau)
        
        filename = str(scenario) + '_' + str(direction)
        
        plt.savefig('./Output/%s_trips.png' % (filename.strip()))
        plt.close()
        

## M_COUNT Peak hours


In [334]:
# parameters
scenarios_ht = [21, 23]
scenarios_names = [u"2016_aht", u"2016_iht"]

scenarios = []
ids = []
modes = []
m_counts = []
m_counts_types = []
voltr = []
directions = []

for i, j in zip(scenarios_ht, scenarios_names):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    scenario = modeller.scenario
    network = scenario.get_network()
    #
    # list @code extra attribuuttiin tallennettuista vaylaosan tunnuksista
    for link in network.links():
        if link['@m_count'] > 0:
            voltr_segment = 0
            for segment in link.segments():
                voltr_segment += segment.transit_volume
            scenarios.append(j)
            ids.append(link.id)
            modes.append(link.modes)
            m_counts.append(link['@m_count'])
            m_counts_types.append(link['@l_count_type'])
            voltr.append(voltr_segment)
            directions.append(link['@direction'])    
    
d = {'scenario': scenarios,
     'id': ids, 
     'l_count': m_counts, 
     'l_count_type': m_counts_types, 
     'mode': [list(x) for x in modes],
     'voltr': voltr, 
     'direction': directions} 
            
df = pd.DataFrame(data = d) 
df.to_csv("m_count_model.txt", index = False, sep='\t')

In [335]:
modes = []
for i in df["mode"].tolist():
    k = ""
    for j in i:
        k = k + str(j)
    modes.append(k)
df["mode"] = modes

df["mode_name"] = ""
df["mode_name"].loc[df["mode"].str.contains("b")] = "bussi"
df["mode_name"].loc[df["mode"].str.contains("m")] = "metro"
df["mode_name"].loc[df["mode"].str.contains("t")] = "raitiotie"
df["mode_name"].loc[df["mode"].str.contains("r")] = "juna"

df = df.merge(l_count_names, on = "l_count_type", how='left')
df = df.merge(directions_names, on = "direction", how='left')

In [336]:
dfPlot = df.groupby(['scenario', 'l_count_name', 'direction_name', 'mode_name'])['l_count', 'voltr'].sum()
dfPlot = dfPlot.reset_index()

In [337]:
image_export_path = os.path.join("./Output/")
if not os.path.exists(image_export_path):
    os.mkdir(image_export_path)

for name in df["l_count_name"].unique():
    for scenario in df["scenario"].unique():
        
        fig, ax = plt.subplots(figsize = [9,5], dpi = 300)

        # set width of bar
        barWidth = 0.3
 
        # set height of bar
        plot_data = dfPlot.loc[dfPlot.scenario == scenario]
        plot_data = plot_data.loc[plot_data.l_count_name == name]
    
        bars_count = plot_data.filter(['l_count']).values
        bars_volau = plot_data.filter(['voltr']).values
        
        
        
        # Set position of bar on X axis
        r1 = np.arange(len(bars_count))
        r2 = [x + barWidth for x in r1]

        # Make the plot
        ax_bars_count = ax.bar(r1, bars_count, color=palette1[0], width=barWidth, edgecolor='white', label=u'Laskenta', alpha = 0.9)
        ax_bars_volau = ax.bar(r2, bars_volau, color=palette1[1], width=barWidth, edgecolor='white', label=u'Mallinnus', alpha = 0.9)
        
        # Add xticks on the middle of the group bars
        
        plt.xticks([r + barWidth*0.5 for r in range(len(bars_count))], plot_data["mode_name"])
        
        plt.tick_params(axis='both', which='major', labelsize=11)
        
        # axis names
        plt.xlim(-0.3, len(bars_count))
        plt.ylim(0, 20000) 
        plt.ylabel(u'Matkustajaa', fontsize = 11)
        
        title = str(scenario).replace('_',' ')
        plt.title(u'Matkustajamäärät %s' % (title),fontweight="bold")
        
        # spines invisible
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        plt.tick_params(top='off', right='off', labelleft='on', labelbottom='on')
        
        # Shrink current axis's height by 10% on the bottom
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.15,
                         box.width, box.height * 0.85])
        
        # Put a legend below current axis
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon= False, fontsize = 11)
        
        def autolabel(bars):
            """
            Attach a text label above each bar displaying its height
            """
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., 1+height,
                        '%d' % int(round(height, -2)),
                        ha='center', va='bottom', color="black", fontsize=11)

        autolabel(ax_bars_count)
        autolabel(ax_bars_volau)
        
        filename = str(scenario) + '_' + str(name)
        
        plt.savefig('./Output/M%s_trips.png' % (filename.strip()))
        plt.close()
        

In [338]:
# parameters
scenarios_ht = [22]
scenarios_names = [u"2016_vrk"]

scenarios = []
ids = []
modes = []
m_counts = []
m_counts_types = []
voltr = []
directions = []

for i, j in zip(scenarios_ht, scenarios_names):
    scenario_selection = modeller.emmebank.scenario(i)
    data_explorer.replace_primary_scenario(scenario_selection)
    scenario = modeller.scenario
    network = scenario.get_network()
    #
    # list @code extra attribuuttiin tallennettuista vaylaosan tunnuksista
    for link in network.links():
        if link['@m_count'] > 0:
            voltr_segment = 0
            for segment in link.segments():
                voltr_segment += segment['@voltr_day']
            scenarios.append(j)
            ids.append(link.id)
            modes.append(link.modes)
            m_counts.append(link['@m_count'])
            m_counts_types.append(link['@l_count_type'])
            voltr.append(voltr_segment)
            directions.append(link['@direction'])    
    
d = {'scenario': scenarios,
     'id': ids, 
     'l_count': m_counts, 
     'l_count_type': m_counts_types, 
     'mode': [list(x) for x in modes],
     'voltr': voltr, 
     'direction': directions} 
            
df = pd.DataFrame(data = d) 
df.to_csv("m_count_model.txt", index = False, sep='\t')

KeyError: '@voltr_day'